In [1]:
import torch
import itertools

import numpy as np
import qiskit.quantum_info as qi

from torch import nn
from codes import swapper, physical_imposition_operator, get_marginals, partialTrace

In [2]:
# For load the model previously trained is necessary to prepare the same architecture used in training stage

Scale = 4

class ConvDenoiser(nn.Module):
    def __init__(self):
        super(ConvDenoiser, self).__init__()

        # Encoder: Downsampling with convolutions and average pooling
        self.encoder1 = nn.Sequential(
            nn.Conv2d(2, Scale * 60, kernel_size = 3, padding = 1),
            nn.Tanh(),
            nn.AvgPool2d(2, 2),

            nn.Conv2d(Scale * 60, Scale * 120, kernel_size = 3, padding = 1),
            nn.Tanh(),
            nn.AvgPool2d(2, 2),

            nn.Conv2d(Scale * 120, Scale * 60, kernel_size = 3, padding = 1),
            nn.Tanh(),
            nn.AvgPool2d(2, 2)
        )

        # Decoder: Upsampling with transpose convolutions
        self.decoder1 = nn.Sequential(
            nn.ConvTranspose2d(Scale * 60, Scale * 60, kernel_size = 3, padding = 1, stride = 2),
            nn.Tanh(),

            nn.ConvTranspose2d(Scale * 60, Scale * 120, kernel_size = 5, padding = 1, stride = 2),
            nn.Tanh(),

            nn.ConvTranspose2d(Scale * 120, Scale * 60, kernel_size = 6, stride = 2),
            nn.Tanh(),

            nn.Conv2d(Scale * 60, 2, kernel_size = 3)  # Final layer to return to 2 channels
        )

    def forward(self, x):
        # Forward pass through encoder and decoder
        encoded = self.encoder1(x)
        output  = self.decoder1(encoded)
        return output

# Instantiate the model
model = ConvDenoiser()

In [3]:
# We load the model from the file checkpoint

use_cpoint      = "./checkpoint_N4_k3"
model_params    = torch.load(use_cpoint)

model.load_state_dict(model_params['model_state_dict'])
model.eval()

ConvDenoiser(
  (encoder1): Sequential(
    (0): Conv2d(2, 240, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): Tanh()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (3): Conv2d(240, 480, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): Tanh()
    (5): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (6): Conv2d(480, 240, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): Tanh()
    (8): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (decoder1): Sequential(
    (0): ConvTranspose2d(240, 240, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): Tanh()
    (2): ConvTranspose2d(240, 480, kernel_size=(5, 5), stride=(2, 2), padding=(1, 1))
    (3): Tanh()
    (4): ConvTranspose2d(480, 240, kernel_size=(6, 6), stride=(2, 2))
    (5): Tanh()
    (6): Conv2d(240, 2, kernel_size=(3, 3), stride=(1, 1))
  )
)

In [4]:
# Define the parameters of the data that's going to be used for test stage

dimension   = 2     # Dimension of each qudits, for qubits is 2.
n_qubits    = 4     # Number of qubits of the global state
n_marginal  = 3     # Number of marginal that we know
n_matrices  = 500  # Number of matrices to be generated per rank

In [5]:
# To define the data is necessary to define a swapper matrix and label the differents marginal

matrix_size = dimension ** n_qubits
subs_dims   = [dimension for _ in range(n_qubits)]
swapper_d   = swapper(dimension)
labels_mg   = list(itertools.combinations(range(n_qubits), r = n_marginal))

rho_noisless    = qi.random_density_matrix(dims = subs_dims).data
test_mg_in      = get_marginals(rho_noisless, dimension, n_qubits, labels_mg)

In [6]:
# Prepare the initial seed to implemente the physical imposition operator and, with that, get a matrix that contains the Quantum Marginals
# but, not necessary, being a Quantum State

initial_seed = qi.random_density_matrix(dims = subs_dims).data
rho_noisy = physical_imposition_operator(dimension,n_qubits, initial_seed, test_mg_in, swapper_d)

print(f"Eigenvals of the noisy matrix =  \n\n   {np.linalg.eigvalsh(rho_noisy)}")
print(40*"---")

Eigenvals of the noisy matrix =  

   [-0.02990364 -0.00653641  0.00549846  0.01349185  0.02386722  0.03313472
  0.04403347  0.05290858  0.06027106  0.06646289  0.08666911  0.0910665
  0.10923285  0.13420924  0.15002503  0.16556904]
------------------------------------------------------------------------------------------------------------------------


In [7]:
# Then, we replace the resulted matrix into our model to convert it into a density matrix

input_state     = torch.Tensor(np.stack((rho_noisy.real, rho_noisy.imag)))
output_state    = model(input_state.reshape(1, 2, matrix_size, matrix_size))
predicted_state = output_state[0][0] + 1j*output_state[0][1]
predicted_state = (predicted_state + torch.conj(predicted_state.T))/2

## Check positivity

The first step here is to check the conditions of the predicted matrix to be a quantum state, that is, check positivity. The normalization always can be done trought divide for the trace so, in general, even if the matrix do not have trace one we can renormalize to impose that.

In [8]:
# We check that the output matrix is a Quantum state, that is, it is trace one and non-negative eigenvalues

predicted_state = predicted_state.detach().numpy()
eigenvalues     = np.linalg.eigvalsh(predicted_state)

print(40*"---")
print(f"Eigenvalues of the predicted state = \n\n  {eigenvalues}\n")

if not np.all(eigenvalues.real >= 0):
    print(f"Warning: negative eigenvalue found {np.min(eigenvalues.real)}")
    
print(40*"---")
print(f"sum of the eigenvals = {sum(eigenvalues)}\n")
print(40*"---")
print(f"Normalization = {np.trace(predicted_state)}\n")
print(40*"---")
print(f"Eigenvals of the generator state =  \n\n   {np.linalg.eigvalsh(rho_noisless)}")
print(40*"---")

------------------------------------------------------------------------------------------------------------------------
Eigenvalues of the predicted state = 

  [0.01543245 0.01752507 0.02005671 0.02542141 0.02706325 0.02842543
 0.03674523 0.03974725 0.07169306 0.07376172 0.07995897 0.08596496
 0.09794374 0.10479727 0.11585578 0.12273619]

------------------------------------------------------------------------------------------------------------------------
sum of the eigenvals = 0.9631284940987825

------------------------------------------------------------------------------------------------------------------------
Normalization = (0.9631284475326538+0j)

------------------------------------------------------------------------------------------------------------------------
Eigenvals of the generator state =  

   [0.00071919 0.00297635 0.00580721 0.00890401 0.01030026 0.0204045
 0.03004263 0.03991976 0.05207595 0.06018732 0.07262291 0.0980476
 0.10790311 0.12619917 0.17895469 0.1

In [9]:
# The original problem is to get the global states that's contain the information about the quantum marginals, that is, if we get the marginals
# from the neural network and, then, we compute the marginals trought partial trace and, then, compute the fidelity between the theoretical
# marginals with the predicted ones the result should be close to 1.

predicted_state = predicted_state/np.trace(predicted_state)
predicted_mg    = get_marginals(predicted_state, dimension, n_qubits, labels_mg)

for k in test_mg_in.keys():
    u, v = test_mg_in[k], predicted_mg[k]
    print(k, qi.state_fidelity(u, v, validate=True))

(0, 1, 2) 0.8645712186604758
(0, 1, 3) 0.8874367430892801
(0, 2, 3) 0.9038057530988741
(1, 2, 3) 0.943073610590342


As we can see above, the actual fidelities are close to one. Better results can be obtained by a more trained model or a model trained with more data.

# Two-body marginals

Suppose a scenario where we just have marginals of two-bodies, that is, we just know quantum states of two-qubits and, with them, we want to compute the global system

In [10]:
# The first step is to label the marginals of the problem, for this case, suppose a marginal from the body 1 and 2.

labels_mg   = [(1, 2)]
test_mg     = get_marginals(rho_noisless, dimension, n_qubits, labels_marginals = labels_mg)
pred_mg     = get_marginals(predicted_state, dimension, n_qubits, labels_marginals = labels_mg)

print(f"Target marginal {list(labels_mg)}: ")
print(np.round(test_mg[labels_mg[0]], decimals=4))

print(f"\nPredicted marginal {list(labels_mg)}:  ")
print(np.round(pred_mg[labels_mg[0]], decimals=4))

print("\nCloseness: ")

print(qi.state_fidelity(pred_mg[labels_mg[0]], test_mg[labels_mg[0]], validate=True))

Target marginal [(1, 2)]: 
[[ 0.2701+0.j     -0.0256-0.0138j  0.0142+0.0276j  0.0194-0.0105j]
 [-0.0256+0.0138j  0.2412-0.j      0.015 -0.0315j  0.0083+0.0087j]
 [ 0.0142-0.0276j  0.015 +0.0315j  0.2266-0.j      0.0053+0.0013j]
 [ 0.0194+0.0105j  0.0083-0.0087j  0.0053-0.0013j  0.2621+0.j    ]]

Predicted marginal [(1, 2)]:  
[[ 0.238 +0.j      0.0103+0.0006j  0.0363+0.001j   0.0236-0.0005j]
 [ 0.0103-0.0006j  0.259 +0.j     -0.0008-0.0004j  0.0365+0.0018j]
 [ 0.0363-0.001j  -0.0008+0.0004j  0.2612+0.j      0.0138+0.0015j]
 [ 0.0236+0.0005j  0.0365-0.0018j  0.0138-0.0015j  0.2418+0.j    ]]

Closeness: 
0.9871977233239974


As we see above, the Closeness computed by the fidelity is close to one, that's means we model correctly predict the quantum global state and, that quantum state, also posses the information of the marginals.